# Stellar Migration

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
from scipy.stats import gaussian_kde

/u/fiza/conda-envs/auriga/lib/python3.7/site-packages/setuptools/_importlib.py:23: UserWarning: `importlib-metadata` version is incompatible with `setuptools`.
This problem is likely to be solved by installing an updated version of `importlib-metadata`.
  warnings.warn(msg)  # Ensure a descriptive message is shown.


In [2]:
from auriga.snapshot import Snapshot
from auriga.images import figure_setup
from auriga.settings import Settings
from auriga.support import find_indices

In [15]:
figure_setup()

In [4]:
settings = Settings()

## Calculate Stellar Migration

In [5]:
def read_data(simulation: str) -> pd.DataFrame:
    s = Snapshot(simulation=simulation, loadonlytype=[0, 1, 2, 3, 4, 5])
    s.tag_in_situ_stars()
    s.tag_particles_by_region(
        disc_std_circ=settings.disc_std_circ,
        disc_min_circ=settings.disc_min_circ,
        cold_disc_delta_circ=settings.cold_disc_delta_circ,
        bulge_max_specific_energy=settings.bulge_max_specific_energy)

    is_real_star = (s.type == 4) & (s.stellar_formation_time > 0)
    is_main_obj = (s.halo == s.halo_idx) & (s.subhalo == s.subhalo_idx)
    is_in_situ = (s.is_in_situ == 1)
    is_target = is_real_star & is_main_obj & is_in_situ

    props = {
        "ID": s.ids[is_target],
        "ComponentTag": s.region_tag[is_target],
        "FormationSnapshot": s.stellar_formation_snapshot[is_target],
        }

    return pd.DataFrame(props)

In [6]:
def read_reduced_data(simulation: str) -> pd.DataFrame:
    s = Snapshot(simulation=simulation, loadonlytype=[0, 1, 2, 3, 4, 5])
    s.tag_particles_by_region(
        disc_std_circ=settings.disc_std_circ,
        disc_min_circ=settings.disc_min_circ,
        cold_disc_delta_circ=settings.cold_disc_delta_circ,
        bulge_max_specific_energy=settings.bulge_max_specific_energy)

    is_real_star = (s.type == 4) & (s.stellar_formation_time > 0)
    is_main_obj = (s.halo == s.halo_idx) & (s.subhalo == s.subhalo_idx)
    is_target = is_real_star & is_main_obj

    props = {
        "ID": s.ids[is_target],
        "ComponentTag": s.region_tag[is_target],
        }

    return pd.DataFrame(props)

In [7]:
with open("../results/stellar_migration.json") as f:
       migration_matrices = json.load(f)

for galaxy in tqdm(settings.galaxies):
    SIMULATION = f"au{galaxy}_or_l4"
    df = read_data(f"{SIMULATION}_s127")

    # Find component tag at birth
    component_at_birth = -1 * np.ones(len(df), dtype=np.int8)
    for i in range(40, 127, 1):
        this_df = read_reduced_data(f"{SIMULATION}_s{i}")

        idx = find_indices(
            this_df["ID"].to_numpy(),
            df["ID"][df["FormationSnapshot"] == i].to_numpy(),
            -1)

        component_at_birth[df["FormationSnapshot"] == i] \
            = this_df["ComponentTag"][idx].to_numpy()

    # Add component tags at birth for the last snapshot
    component_at_birth[df["FormationSnapshot"] == 127] \
            = df["ComponentTag"][df["FormationSnapshot"] == 127].to_numpy()

    df["ComponentTagAtBirth"] = component_at_birth

    migration_matrix = np.zeros((4, 4))
    for i in range(0, 4):
        n_particles = (df["ComponentTag"] == i).sum()
        for j in range(0, 4):
            migration_matrix[i, j] = (
                (df["ComponentTag"] == i) \
                    & (df["ComponentTagAtBirth"] == j)).sum() \
                        / n_particles * 100
    
    migration_matrices[SIMULATION] = list(migration_matrix.flatten())

    with open("../results/stellar_migration.json", "w") as f:
        json.dump(migration_matrices, f)

## Results

In [8]:
# Read data
with open("../results/stellar_migration.json", 'r') as f:
    migration_matrices = json.load(f)

### Plot Galaxy

In [9]:
SIMULATION = "au6_or_l4"

In [10]:
migration_matrix = np.array(migration_matrices[SIMULATION]).reshape(4, 4)

In [55]:
fig = plt.figure(figsize=(3.0, 3.0))
gs = fig.add_gridspec(nrows=1, ncols=1, hspace=0.0, wspace=0.0)
ax = gs.subplots(sharex=True, sharey=False)

ax.matshow(migration_matrix, cmap="viridis", vmin=0, vmax=100)

ax.set_xlabel(r"Component at Birth")
ax.set_xticks([0, 1, 2, 3])
ax.set_xticklabels(settings.components)
ax.tick_params(axis='x', bottom=True, top=False,
               labelbottom=True, labeltop=False)

ax.set_ylabel(r"Component at $z=0$")
ax.set_yticks([0, 1, 2, 3])
ax.set_yticklabels(settings.components)

for i in range(migration_matrix.shape[0]):
    for j in range(migration_matrix.shape[1]):
        color = "white" if migration_matrix[j, i] < 50 else "black"
        ax.text(
            i, j,
            r"$\mathbf{" + f"{np.round(migration_matrix[j, i], 1)}" + r"\%}$",
            c=color, ha="center", va="center")

ax.text(x=0.01, y=1.01, size=7.0,
        s=r"$\texttt{" + f"{SIMULATION}".upper() + "}$",
        ha="left", va="bottom", transform=ax.transAxes)

fig.savefig(f"../images/stellar_migration/{SIMULATION}.pdf")
plt.close(fig)

### Averages

In [12]:
# Turn dictionary into array
migration_matrices_arr = np.nan * np.ones((31, 4, 4))
for i in settings.galaxies:
    migration_matrices_arr[i] = np.array(
        migration_matrices[f"au{i}_or_l4"]).reshape(4, 4)

In [13]:
groups = [[i for i in range(1, 31)],
          settings.groups["MilkyWayLike"],
          settings.groups["NotMilkyWayLike"],
          settings.groups["InsideOut"],
          settings.groups["NotInsideOut"]]
names = ["All", "G1", "G2", "IO", "OI"]
colors = ["black", "tab:green", "tab:red", "tab:purple", "tab:blue"]

In [54]:
fig = plt.figure(figsize=(7.2, 7.2))
gs = fig.add_gridspec(nrows=4, ncols=4, hspace=0.0, wspace=0.0)
axs = gs.subplots(sharex=True, sharey=False)

for ax in axs.flatten():
    ax.set_xlim(0, 100)
    ax.set_xticks([20, 40, 60, 80])
    ax.set_xlabel(r"$f_\star$ [\%]")
    ax.set_ylim(0, 0.14)
    ax.set_yticks([0.02, 0.04, 0.06, 0.08, 0.10, 0.12])
    ax.set_ylabel("PDF")
    ax.label_outer()

for i in range(len(settings.components)):
    for j in range(len(settings.components)):
        title = r"$\textbf{" + settings.components[j] + r"}_\mathrm{Birth} \to \textbf{" \
            + settings.components[i] + r"}_\mathrm{Today}$"
        axs[i, j].text(x=0.5, y=0.95, s=title,
                       size=7.5, transform=axs[i, j].transAxes,
                       ha="center", va="top")
        for k, group in enumerate(groups):
            axs[i, j].hist(migration_matrices_arr[group, i, j],
                           range=(0, 100), bins=10, density=True, lw=1.5,
                           histtype="step", color=colors[k], label=names[k])

axs[0, 0].legend(loc="center right", framealpha=0, fontsize=7.0)

fig.savefig(f"../images/stellar_migration/statistics.pdf")
plt.close(fig)